In [34]:
import numpy as np
import tensorflow as tf
import cv2
import time
import os
from collections import deque, Counter

last_capture_time = time.time()
counter = 0
# Preprocessing function for each frame
def preprocess_frame(frame, target_size=(112, 112)):
    resized_frame = cv2.resize(frame, target_size)
    return resized_frame.astype(np.float32) / 255.0

def most_frequent_label(predictions_deque):
    if not predictions_deque:
        return "Unknown"
    label_counts = Counter(predictions_deque)
    most_common_label, count = label_counts.most_common(1)[0]
    return most_common_label

def predict_live_video(model, video_source, target_size=(112, 112), frame_count=10, capture_interval=0.1):
    global last_capture_time, counter
    predicted_class = 2
    cap = cv2.VideoCapture(video_source)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 360)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 100)
    
    if not cap.isOpened():
        print(f"Error: Could not open video source: {video_source}")
        return
    
    frames = []
    slowdown_factor = 1
    fps = cap.get(cv2.CAP_PROP_FPS)  # Get original FPS
    delay = (1 / fps) * slowdown_factor  # Slow-motion delay

    print("Starting prediction... Press 'q' to exit.")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("End of video stream or error reading the frame.")
            break

        

        current_time = time.time()
        elapsed_time = current_time - last_capture_time

        # If enough time has passed, process the frame
        #if elapsed_time >= capture_interval:
        processed_frame = preprocess_frame(frame, target_size)
        frames.append(processed_frame)
        last_capture_time = current_time  # Reset the capture time

        if len(frames) == frame_count:
            batch_frames = np.expand_dims(np.stack(frames, axis=0), axis=0)

            # This is the confidence probability
            prediction = model.predict(batch_frames, verbose=0)

            # Get and Output the one with Highest Probability
            predicted_class = np.argmax(prediction, axis=1)[0]

            frames = []
            #print("Prediction {}:".format(counter))
            #print(predicted_class)
            counter += 1

        # Get the most frequent label from the history
        stable_prediction = predicted_class

        # Map class index to labels
        if stable_prediction == 0:
            label = "Brawl"
        elif stable_prediction == 1:
            label = "Fainting"
        elif stable_prediction == 2:
            label = "Normal"
        elif stable_prediction == 3:
            label = "Panic"
        else:
            label = "Unknown"

        # Overlay label on the current frame
        cv2.putText(frame, f"Prediction: {label}", (20, 30), cv2.FONT_HERSHEY_SIMPLEX,
                    1, (0, 255, 0), 2)

        for _ in range(slowdown_factor):
            cv2.imshow("Real-time Monitoring", frame)
            time.sleep(delay)  # Introduce delay

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [41]:
model_path = 'Project_Model/second_chance15.keras'
model = tf.keras.models.load_model(model_path)
print("Model loaded successfully!")
print(model.input_shape)

Model loaded successfully!
(None, 16, 112, 112, 3)


In [42]:
video_path = "D:/Virtual_Storage/JIN/test_video/test4.mp4"

if os.path.isfile(video_path) and video_path.lower().endswith(('.avi', '.mpeg', '.mp4')):
    predict_live_video(model, video_path, frame_count=16, capture_interval=0.2)
else:
    print(f"Error: The file '{video_path}' does not exist or is not a valid video format.")

Starting prediction... Press 'q' to exit.
